<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/notebook_util/generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [1]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Configure dataset
#@markdown - choose whether to use wandb id or use checkpoint_path to select checkpoint file

#@markdown Choice 1: wandb id and project_name to select checkpoint file
#@markdown - set `"run_id"` to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
id = "3coc81fr" #@param {type:"string"}
#@markdown Enter project name (either `chair-gan`, `handtool-gan` or `tree-gan`)
project_name = "tree-gan" #@param ["tree-gan", "handtool-gan", "chair-gan"]

#@markdown Choice 2: file path and model type to select saved checkpoint .ckpt file
#@markdown - For example via the file browser on the left to locate and right click to copy the path.
#@markdown - file path example: `/content/drive/My Drive/h/k/checkpoint.ckpt` 
ckpt_file_location = "" #@param {type:"string"}
#@markdown - Enter trained neural network model type
#@markdown - (may be necessary for wandb_id if selected_model is not saved in config)
selected_model = "VAEGAN"    #@param ["VAEGAN", "GAN", "VAE", "WGAN", "WGAN_GP"]


#@markdown Enter how many samples to generate
test_num_samples = 20    #@param {type:"integer"}
#@markdown Enter export location (folder/directory).    
export_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}" #@param {type:"string"}



if id and ckpt_file_location:
    raise Exception("Only one of id / ckpt_file_location can be set!")
if (not id) and (not ckpt_file_location):
    raise Exception("Please set id / ckpt_file_location!")

if id:
    print("id:", id)
    print("project_name:", project_name)
else:
    print("selected_model:", selected_model)
    print("ckpt_file_location:", ckpt_file_location)
print("test_num_samples:", test_num_samples)
print("export_location:", export_location)


id: 3coc81fr
project_name: tree-gan
test_num_samples: 20
export_location: /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/3coc81fr


In [3]:
from argparse import Namespace, ArgumentParser
#@markdown Install wandb and log in
entity="bugan"
rev_number = None
if id:
    !pip install wandb
    output.clear()
    #find wandb API key file to auto login
    import wandb
    wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
    wandb_local_netrc_path = Path("/root/.netrc")
    if wandb_drive_netrc_path.exists():
        import shutil

        print("Wandb .netrc file found, will use that to log in.")
        shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
    else:
        print(
            f"Wandb config not found at {wandb_drive_netrc_path}.\n"
            f"Using manual login.\n\n"
            f"To use auto login in the future, finish the manual login first and then run:\n\n"
            f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
            f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
            f"Then that file will be used to login next time.\n"
        )

    !wandb login

    #read information (run config, etc) stored online
        #all config will be replaced by the stored one in wandb
    api = wandb.Api()
    previous_run = api.run(f"{entity}/{project_name}/{id}")
    config = Namespace(**previous_run.config)
        #load selected_model, rev_number in the config
    if hasattr(config, "selected_model"):
        selected_model = config.selected_model
    if hasattr(config, "rev_number"):
        rev_number = config.rev_number

    run = wandb.init(project=project_name, id=id, entity=entity, resume=True, dir="./", group=selected_model)

    output.clear()
    print("run id: " + str(wandb.run.id))
    print("run name: " + str(wandb.run.name))
    print("ok!")

run id: 3coc81fr
run name: bumbling-glitter-744
ok!


# To just train a model, no edits should be required in any cells below.

In [4]:
import os
import io
import sys
from pathlib import Path
os.environ["WANDB_MODE"] = "dryrun"

%cd /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/

run_path = "./"

!apt-get update

!pip install pytorch-lightning
!pip install trimesh
!apt install -y xvfb
!pip install trimesh xvfbwrapper
#bugan package
if rev_number is not None:
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git@{rev_number}#egg=bugan
else:
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git#egg=bugan

import bugan
#EXTRACT package version
    #switch stdout to temperary stringIO
old_stdout = sys.stdout
temp_stdout = io.StringIO()
sys.stdout = temp_stdout
    #get version
%pip freeze | grep bugan
version = temp_stdout.getvalue()
rev_number = version.split("+g")[1].rstrip()
    #switch back stdout
sys.stdout = old_stdout

output.clear()
print("bugan package revision number: " + str(rev_number))
print('ok!')

bugan package revision number: 083aaf0
ok!


In [5]:
import io
from io import BytesIO
import zipfile
import trimesh
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


from bugan.functionsPL import *
from bugan.modelsPL import *

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from xvfbwrapper import Xvfb

#generate samples

In [6]:
if id:
    checkpoint_path = os.path.join(wandb.run.dir, 'checkpoint.ckpt')
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
else:
    checkpoint_path = ckpt_file_location

#model
if selected_model == "VAEGAN":
    MODEL_CLASS = VAEGAN
elif selected_model == "GAN":
    MODEL_CLASS = GAN
elif selected_model == "VAE":
    MODEL_CLASS = VAE_train
elif selected_model == "WGAN":
    MODEL_CLASS = GAN_Wloss
else:
    MODEL_CLASS = GAN_Wloss_GP

model = MODEL_CLASS.load_from_checkpoint(checkpoint_path)
model = model.eval().to(device)
sample_trees = model.generate_tree(num_trees=test_num_samples)

#create directory
export_location = Path(export_location)
if not export_location.exists():
    export_location.mkdir(parents=True)

sample_meshes = []
for n in range(test_num_samples):
    sample_tree_array = sample_trees[n]
    voxelmesh = netarray2mesh(sample_tree_array)
    export_path = export_location / f"sample_{n}.obj"
    voxelmesh.export(file_obj=export_path, file_type="obj")
print("ok!")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


ok!
